# <center> Предсказание победителя в Dota 2
<center> <img src="https://meduza.io/impro/YnJZAHUW6WHz_JQm1uRPkTql_qAhbfxt3oFJLGH7CJg/fill/980/0/ce/1/aHR0cHM6Ly9tZWR1/emEuaW8vaW1hZ2Uv/YXR0YWNobWVudHMv/aW1hZ2VzLzAwNy8x/NTcvNjk1L29yaWdp/bmFsL0tMVThLbUti/ZG5pSzlibDA0Wmlw/WXcuanBn.webp" width="700" height="700">

[Почитать подбробнее](https://meduza.io/feature/2021/10/19/rossiyskaya-komanda-vyigrala-chempionat-mira-po-dota-2-i-poluchila-18-millionov-dollarov-postoyte-otkuda-takie-dengi-neuzheli-igrat-v-dotu-tak-slozhno)

#### [Оригинальная статья](https://arxiv.org/pdf/2106.01782.pdf)
    
### Начало

Посмотрим на готовые признаки и сделаем первую посылку. 

1. [Описание данных](#Описание-данных)
2. [Описание признаков](#Описание-признаков)
3. [Наша первая модель](#Наша-первая-модель)
4. [Посылка](#Посылка)

### Первые шаги на пути в датасайенс

5. [Кросс-валидация](#Кросс-валидация)
6. [Что есть в json файлах?](#Что-есть-в-json-файлах?)
7. [Feature engineering](#Feature-engineering)

### Импорты

In [1]:
import os
import json
import pandas as pd
import datetime
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

%matplotlib inline

In [2]:
SEED = 10801
sns.set_style(style="whitegrid")
plt.rcParams["figure.figsize"] = 12, 8
warnings.filterwarnings("ignore")

## <left>Описание данных

Файлы:

- `sample_submission.csv`: пример файла-посылки
- `train_raw_data.jsonl`, `test_raw_data.jsonl`: "сырые" данные 
- `train_data.csv`, `test_data.csv`: признаки, созданные авторами
- `train_targets.csv`: результаты тренировочных игр

## <left>Описание признаков
    
Набор простых признаков, описывающих игроков и команды в целом

In [3]:
PATH_TO_DATA = "../input/bi-2021-ml-competitions-dota2"

df_train_features = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                             "train_data.csv"), 
                                    index_col="match_id_hash")
df_train_targets = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                            "train_targets.csv"), 
                                   index_col="match_id_hash")

In [ ]:
df_train_features.shape

In [ ]:
df_train_features.head()

Имеем ~32 тысячи наблюдений, каждое из которых характеризуется уникальным `match_id_hash` (захэшированное id матча), и 245 признаков. `game_time` показывает момент времени, в который получены эти данные. То есть по сути это не длительность самого матча, а например, его середина, таким образом, в итоге мы сможем получить модель, которая будет предсказывать вероятность победы каждой из команд в течение матча (хорошо подходит для букмекеров).

Нас интересует поле `radiant_win` (так называется одна из команд, вторая - dire). Остальные колоки здесь по сути получены из "будущего" и есть только для тренировочных данных, поэтому на них можно просто посмотреть).

In [ ]:
df_train_targets.head()

## <left>Наша первая модель

In [31]:
X = df_train_features.values
y = df_train_targets["radiant_win"].values.astype("int8")

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.3, 
                                                      random_state=SEED)

#### Обучим случайный лес

In [ ]:
%%time
rf_model = RandomForestClassifier(n_estimators=300, max_depth=7, n_jobs=-1, random_state=SEED)
rf_model.fit(X_train, y_train)

#### Сделаем предсказания и оценим качество на отложенной части данных

In [ ]:
y_pred = rf_model.predict_proba(X_valid)[:, 1]

In [ ]:
valid_score = roc_auc_score(y_valid, y_pred)
print("ROC-AUC score на отложенной части:", valid_score)

Посмотрим на accuracy:

In [ ]:
valid_accuracy = accuracy_score(y_valid, y_pred > 0.5)
print("Accuracy score (p > 0.5) на отложенной части:", valid_accuracy)

## <left>Посылка

In [4]:
df_test_features = pd.read_csv(os.path.join(PATH_TO_DATA, "test_data.csv"), 
                                   index_col="match_id_hash")

X_test = df_test_features.values

In [5]:
y_test_pred = rf_model.predict_proba(X_test)[:, 1]

df_submission = pd.DataFrame({"radiant_win_prob": y_test_pred}, 
                                 index=df_test_features.index)

In [ ]:
submission_filename = "submission_{}.csv".format(
    datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
df_submission.to_csv(submission_filename)
print("Файл посылки сохранен, как: {}".format(submission_filename))

## <left>Кросс-валидация

Во многих случаях кросс-валидация оказывается лучше простого разбиения на test и train. Воспользуемся `ShuffleSplit` чтобы создать 5 70%/30% наборов данных.

In [ ]:
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=SEED)

In [ ]:
%%time
rf_model = RandomForestClassifier(n_estimators=300, max_depth=7, n_jobs=-1, random_state=SEED)
cv_scores_rf = cross_val_score(rf_model, X, y, cv=cv, scoring="roc_auc")

In [ ]:
cv_scores_rf

In [ ]:
print(f"Среднее значение ROC-AUC на кросс-валидации: {cv_scores_rf.mean()}")

## <left>Что есть в json файлах?

Описание сырых данных можно найти в `train_matches.jsonl` и `test_matches.jsonl`. Каждый файл содержит одну запись для каждого матча в [JSON](https://en.wikipedia.org/wiki/JSON) формате. Его легко превратить в питоновский объект при помощи метода `json.loads`.

In [ ]:
with open(os.path.join(PATH_TO_DATA, "train_raw_data.jsonl")) as fin:
    # прочтем 419 строку
    for i in range(419):
        line = fin.readline()
    
    # переведем JSON в питоновский словарь 
    match = json.loads(line)

In [ ]:
match['objectives']

In [ ]:
player = match["players"][9]
player["kills"], player["deaths"], player["assists"]

KDA - может быть неплохим признаком, этот показатель считается как:
    
<center>$KDA = \frac{K + A}{D}$

Информация о количестве использованных способностей:

In [ ]:
player["ability_uses"]

In [ ]:
for i, player in enumerate(match["players"]):
    plt.plot(player["times"], player["xp_t"], label=str(i+1))

plt.legend()
plt.xlabel("Time, s")
plt.ylabel("XP")
plt.title("XP change for all players");

#### Сделаем чтение файла с сырыми данными и добавление новых признаков удобным

В этот раз для чтение `json` файлов лучше использовать библиотеку `ujson`, иначе все будет слишком долго :(

In [5]:
try:
    import ujson as json
except ModuleNotFoundError:
    import json
    print ("Подумайте об установке ujson, чтобы работать с JSON объектами быстрее")
    
try:
    from tqdm.notebook import tqdm
except ModuleNotFoundError:
    tqdm_notebook = lambda x: x
    print ("Подумайте об установке tqdm, чтобы следить за прогрессом")

    
def read_matches(matches_file, total_matches=31698, n_matches_to_read=None):
    """
    Аргуент
    -------
    matches_file: JSON файл с сырыми данными
    
    Результат
    ---------
    Возвращает записи о каждом матче
    """
    
    if n_matches_to_read is None:
        n_matches_to_read = total_matches
        
    c = 0
    with open(matches_file) as fin:
        for line in tqdm(fin, total=total_matches):
            if c >= n_matches_to_read:
                break
            else:
                c += 1
                yield json.loads(line)

#### Чтение данных в цикле

Чтение всех данных занимает 1-2 минуты, поэтому для начала можно попробовать следующее:

1. Читать 10-50 игр
2. Написать код для работы с этими JSON объектами
3. Убедиться, что все работает
4. Запустить код на всем датасете
5. Сохранить результат в `pickle` файл, чтобы в следующий раз не переделывать все заново

## <left>Feature engineering

Напишем функцию, которая поможет нам легче добавлять новые признаки.

In [6]:
def add_new_features(df_features, matches_file):
    """
    Аргуенты
    -------
    df_features: таблица с данными
    matches_file: JSON файл с сырыми данными
    
    Результат
    ---------
    Добавляет новые признаки в таблицу
    """
    
    for match in read_matches(matches_file):
        match_id_hash = match['match_id_hash']

        # Посчитаем количество разрушенных вышек обеими командами
        radiant_tower_kills = 0
        dire_tower_kills = 0
        for objective in match["objectives"]:
            if objective["type"] == "CHAT_MESSAGE_TOWER_KILL":
                if objective["team"] == 2:
                    radiant_tower_kills += 1
                if objective["team"] == 3:
                    dire_tower_kills += 1
        
        

        df_features.loc[match_id_hash, "radiant_tower_kills"] = radiant_tower_kills
        df_features.loc[match_id_hash, "dire_tower_kills"] = dire_tower_kills
        df_features.loc[match_id_hash, "diff_tower_kills"] = radiant_tower_kills - dire_tower_kills
        
        # ... (/¯◡ ‿ ◡)/¯☆*:・ﾟ добавляем новые признаки ...

In [9]:
# Скопируем таблицу с признаками
df_train_features_extended = df_train_features.copy()

# Добавим новые
add_new_features(df_train_features_extended, 
                 os.path.join(PATH_TO_DATA, 
                              "train_raw_data.jsonl"))

In [7]:
df_test_extended = df_test_features.copy()

add_new_features(df_test_extended,os.path.join(PATH_TO_DATA,
                                               "test_raw_data.jsonl"))

In [10]:
df_train_features_extended.to_csv('df_train_extended.csv', index = True)
df_test_extended.to_csv('df_test_extended.csv', index = True)


In [21]:
df_train_features_extended = pd.read_csv('../input/dota2-competition-bi2022/df_train_extended.csv', index_col='match_id_hash')

In [ ]:
df_train_features_extended.head()

In [ ]:
%%time
cv_scores_base = cross_val_score(rf_model, X, y, cv=cv, scoring="roc_auc", n_jobs=-1)
cv_scores_extended = cross_val_score(rf_model, df_train_features_extended.values, y, 
                                     cv=cv, scoring="roc_auc", n_jobs=-1)

In [ ]:
print(f"ROC-AUC на кросс-валидации для базовых признаков: {cv_scores_base.mean()}")
print(f"ROC-AUC на кросс-валидации для новых признаков: {cv_scores_extended.mean()}")

Видно, что случайный лес стал работать немного лучше при добавлении новых признаков. A еще нужно, наверное, как-то по-умному закодировать категориальные признаки.

Дальше дело за малым. Добавляйте новые признаки, пробуйте другие методы, которые мы изучили, а также что-то интересное, что мы не прошли. Удачи!

Перед тем, как будем модифцировать датасет и подбирать модель, пробовать другие алгоритмы, еще раз взглянем на наш датасет. Особенно инетересно, сколько среди наших фичей выосок скоррелированных?

In [ ]:
#Lets see which variables do we have now 
#(suppose that players in both comands have the same complect of features)
[col for col in df_train_features_extended.columns if col.startswith('r1')]

In [ ]:
# Check if any features correlate between each other

for team in ['r1', 'd1']:
    t = [col for col in df_train_features_extended.columns if col.startswith(team)]
    t_train = df_train_features_extended[t].select_dtypes(include=['int64','float64'])
    cor_t = t_train.corr()
    plt.figure()
    sns.heatmap(cor_t)
    if team == 'r1':
        plt.title('Radiant correlation heatmap of features')
    else:
        plt.title('Dire correlation heatmap of features')

Если посмотреть на корреляцию между фичами для двух команд, то можно увидеть, что в обоих случаях у нас есть сильно коррелирующие фичи. Высоко скоррелированные лучше заменять на общую переменную или исключать, так как такие признаки могут привести к переобучению модели. Особенно, если мы в итоге будем использовать ансамбли с деревьями (то есть Random Forest). Среди такие фичей сильно скоррелированными оказались:
1. Золото
2. Опыт
3. Число убитых, смертей и "поддержек (?)"
4. Общее количество здоровья, максимальные единицы здоровья и максимум маны
5. Уровень 

В целом, это не удивительно. Так как действительно, набор опыта зависит от количества убитых персонажей, собранного золота. А от опыта зависит уровень. А от уровня - единицы здоровья и маны (хотя вообще еще от типа персонажа).

Для дальнейшего конструирования модели имело бы смысл объединить/заменить, например, число убитый персонажей, смертей и поддержек. А также объединить золото и опыт в одну переменную. Кроме этого, создав новую, которая бы отражала разницу между золотом и опытом у двух команд. Сначала попробуем разность.

В целом такие переменные, как единицы здоровья и маны с меньшей вероятностью могут иметь корреляцию с тем, победит или проиграет команда (так как не всегда команда, у которой больше запас здоровья и больше израсходована мана оказывается победителем над обратной). Потому эти переменные попрубем в целом исключить из датасета.

Уровнь персонажа может оказывать влияние на то, какая команда выиграет. Так как игроки с более высоким уровнем в целом могут показывать лучшее качество игры во время матча, что также может сказаться на благополучии матча для команды.

Попробуем ввести новую переменную: KDA.

Но прежде проверим распределения наших данных для понимания необходимости стандартизации. Возьмем только численные признаки.

In [12]:

bad_features = ['health', 'max_health', 'max_mana', 'x', 'y', 'hero_id']
not_numeric = ['game_time','game_mode', 'lobby_type', 'objectives_len', 'chat_len']
for bad in bad_features:
    r_col = [f"r{i}_{bad}" for i in range(1,6)]
    d_col = [f"d{i}_{bad}" for i in range(1,6)]
    not_numeric.extend(r_col)
    not_numeric.extend(d_col)



In [18]:
df_train_features_extended.columns

In [22]:
df_train_features_extended.drop(not_numeric, axis = 1).describe()
df_test_extended.drop(not_numeric, axis = 1)

Мы видим, что наши данные имеют очень разные стандартные отклонения, что сказывается на качестве предсказания модели, когда среди всего пула признаков общаа дисперсия не валидна для каждого признака отдельно. Попробуем стандартизовать наши данные.

In [23]:
num_cols = list(df_train_features_extended.drop(not_numeric, axis = 1).columns)

In [24]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
num_arr = scaler.fit_transform(df_train_features_extended[num_cols])
num_df_feat = pd.DataFrame(num_arr, columns=num_cols, index = df_train_features_extended.index)
num_df_feat.head(3)

In [25]:
test = df_train_features_extended.copy()
test2 = pd.concat([test[not_numeric], num_df_feat], axis = 1)

In [14]:
test2[num_cols].describe()

Теперь лучше и наши численные переменные имеют примерно одинаковые распределения. Повторим для тестовых данных.

In [16]:
from sklearn.preprocessing import StandardScaler
num_cols = list(df_test_extended.drop(not_numeric, axis = 1).columns)
scaler = StandardScaler()
num_arr = scaler.fit_transform(df_test_extended[num_cols])
num_df_feat = pd.DataFrame(num_arr, columns=num_cols, index = df_test_extended.index)
df_test_2 = pd.concat([df_test_extended[not_numeric], num_df_feat], axis = 1)

In [17]:
def KDA_calc(data, team):
    kills, deaths, assist = 0, 0, 0
    for player in range(1,6):
        cur = f"{team}{player}"
        kills += data[f"{cur}_kills"]
        deaths += data[f"{cur}_deaths"]
        assist += data[f"{cur}_assists"]
    
    kda = ((kills + assist) / deaths) 
    
    return kda

In [57]:
test2.KDA_radiant

In [26]:
for match in list(test2.index):
    test2.loc[match, 'KDA_radiant'] = KDA_calc(test2.loc[match,], team = 'r')
    test2.loc[match, 'KDA_dire'] = KDA_calc(test2.loc[match,], team = 'd')
for match in list(df_test_2.index):
    df_test_2.loc[match, 'KDA_radiant'] = KDA_calc(df_test_2.loc[match,], team = 'r')
    df_test_2.loc[match, 'KDA_dire'] = KDA_calc(df_test_2.loc[match,], team = 'd')
    
#list(df_train_features_extended.index)

In [40]:
test2.head()

Посмотрим, сколько у нас нулевых и пропущенных значений в данных. Только из первых 5 матчей видно, что тот же KDA у нас для некоторых не определен. Скорее, это связано с делением на 0 смертей, осуществленных командой. Посмотрим на наши данные еще внимательнее.

In [27]:
data = test2.copy()
data_tst = df_test_2.copy()

In [43]:
null = data.isnull().sum()
null[null > 0.0]

Мы видим, что только для KDA у нас присутствуют NaN в датафрейме. Что можно сделать? Попробуем заменить их на среднее. По крайней мере, это первое, что может помочь в данной ситуации. Можно было бы и удалить такие матчи, однако это в целом внесет значимые байесы и не позволит обучить модель определять исход даже при таких данных ( а в целом команды часто могут никого не убить ).

In [28]:
import numpy as np
data.KDA_radiant.replace(np.inf, np.nan, inplace = True)
data.KDA_dire.replace(np.inf, np.nan, inplace = True)

data = data.fillna({'KDA_radiant': data.KDA_radiant.mean(),
                    'KDA_dire': data.KDA_dire.mean()})

data['KDA_diff'] = data.KDA_radiant - data.KDA_dire

data_tst.KDA_radiant.replace(np.inf, np.nan, inplace = True)
data_tst.KDA_dire.replace(np.inf, np.nan, inplace = True)

data_tst = data_tst.fillna({'KDA_radiant': data_tst.KDA_radiant.mean(),
                    'KDA_dire': data_tst.KDA_dire.mean()})

data_tst['KDA_diff'] = data_tst.KDA_radiant - data_tst.KDA_dire

Функция для подсчета метрик команды по всем игрокам

In [29]:
def var_calc(data, team, feature, fun = 'sum'):
    columns = [f"{team}{i}_{feature}" for i in range(1,6)]
    if fun == 'mean':
        var = data[columns].mean(1).astype("float32")
    elif fun == 'std':
        var = data[columns].std(1).astype("float32")
    else:
        var = data[columns].sum(1).astype("float32")
        
    return var

In [30]:
drop_feat = ['health', 'max_health', 'max_mana', 'x', 'y']

In [31]:
player_feat = ['kills', 'deaths', 'assists', 'denies', 'gold', 'lh', 'xp', 'level', 'stuns', 'creeps_stacked', 'camps_stacked', 'rune_pickups',
          'firstblood_claimed', 'teamfight_participation', 'towers_killed', 'roshans_killed', 'obs_placed', 'sen_placed']

Добавим новые фичи, которые описывают общее, среднее и стандартное отклонение по каждой из списка для команды, а также разность общего скора по фиче между командами. Так мы попробуем выделить тренд, которые позволит вероятностно оценивать выигрыш той/иной команды.

In [32]:
for feature in player_feat:
    # train data
    data[f"radiant_{feature}"] = var_calc(data, team = 'r', feature = feature)
    data[f"radiant_{feature}_mean"] = var_calc(data, team = 'r', feature = feature, fun = 'mean')
    data[f"radiant_{feature}_std"] = var_calc(data, team = 'r', feature = feature, fun = 'std')
    
    data[f"dire_{feature}"] = var_calc(data, team = 'd', feature = feature)
    data[f"dire_{feature}_mean"] = var_calc(data, team = 'd', feature = feature, fun = 'mean')
    data[f"dire_{feature}_std"] = var_calc(data, team = 'd', feature = feature, fun = 'std')
    
    data[f"{feature}_diff"] = data[f"radiant_{feature}"] - data[f"dire_{feature}"]
    data[f"{feature}_mean_diff"] = data[f"radiant_{feature}_mean"] - data[f"dire_{feature}_mean"]
    data[f"{feature}_std_diff"] = data[f"radiant_{feature}_std"] - data[f"dire_{feature}_std"]
    
    # test data
    data_tst[f"radiant_{feature}"] = var_calc(data_tst, team = 'r', feature = feature)
    data_tst[f"radiant_{feature}_mean"] = var_calc(data_tst, team = 'r', feature = feature, fun = 'mean')
    data_tst[f"radiant_{feature}_std"] = var_calc(data_tst, team = 'r', feature = feature, fun = 'std')
    
    data_tst[f"dire_{feature}"] = var_calc(data_tst, team = 'd', feature = feature)
    data_tst[f"dire_{feature}_mean"] = var_calc(data_tst, team = 'd', feature = feature, fun = 'mean')
    data_tst[f"dire_{feature}_std"] = var_calc(data_tst, team = 'd', feature = feature, fun = 'std')
    
    data_tst[f"{feature}_diff"] = data_tst[f"radiant_{feature}"] - data_tst[f"dire_{feature}"]
    data_tst[f"{feature}_mean_diff"] = data_tst[f"radiant_{feature}_mean"] - data_tst[f"dire_{feature}_mean"]
    data_tst[f"{feature}_std_diff"] = data_tst[f"radiant_{feature}_std"] - data_tst[f"dire_{feature}_std"]

Уберем, как кажется, незначащие переменные.

In [33]:
for bad in drop_feat:
    r_col = [f"r{i}_{bad}" for i in range(1,6)]
    d_col = [f"d{i}_{bad}" for i in range(1,6)]
    data.drop(r_col, axis = 1, inplace = True)
    data.drop(d_col, axis = 1, inplace = True)
    
    data_tst.drop(r_col, axis = 1, inplace = True)
    data_tst.drop(d_col, axis = 1, inplace = True)

In [24]:
null = data.isnull().sum()
null[null != 0]

In [27]:
data

Попробуем использовать ту же модель, что и раньше, чтобы оценить, как с добавлением новых признаком  у нас изменилось качество предсказания.

In [28]:
rf_model = RandomForestClassifier(n_estimators=300, max_depth=7, n_jobs=-1, random_state=SEED)

In [29]:
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=SEED)

In [33]:
cv_scores_base = cross_val_score(rf_model, X, y, cv=cv, scoring="roc_auc", n_jobs=-1)
cv_scores_extended = cross_val_score(rf_model, data, y, 
                                     cv=cv, scoring="roc_auc", n_jobs=-1)

In [34]:
cv_scores_extended.mean()

Мы видим, что еще до оптимизации модели нам удалось за счет энжиниринга фичей повысить скор до 0.81. В целом, можно уже приступать к подбору параметров и модели для предсказания победы, а можно еще поиграть с категориальными признаками: id героя. И также "почистить" данные от лишних признаков (режим игры, длина чата, длина objectives, тип лобби).

Попробуем сделать кодировку категориальных признаков.

In [34]:
extended_feat = ['game_mode', 'lobby_type', 'objectives_len', 'chat_len']

In [35]:
data2 = data.copy()
data_tst2 = data_tst.copy()
for extended in extended_feat:
    data2.drop(extended, axis = 1, inplace = True)
    data_tst2.drop(extended, axis = 1, inplace = True)


Перед лейблингом категориальных признаков посмотрим, а какие вообще фичи имеют наибольший вклад в точность предсказания выигрыша?

Для этого создадим обучающую функцию и тренировочный, тестовый дадасеты. Важно будет дропнуть индексы, иначе при кросс-валидации не получится рандомно выбирать по фичам.

In [54]:
#  Check if any Nan, None values exist
null = data2.isnull().sum()
null[null != 0]

Функция для тренировки моделей и оценки вклада фичей будет иметь возможность запуска алгоритма классификации. Оценка вклада фичей будет через метод LGB. Во-первых, по ресурсоемкости он показал себя как наиболее оптимальный при работе с большими датасетами (большими относительно количества фичей), занимая минимальное количество памяти, а также CPU. Во-вторых, так как в отличие от других ансамблей, основанных на деревьях, этот метод при классификации растет не в ширину (что повышает вероятность переобучить и поделить слишком сильно, когда в каждой ветви буквально по одному измерению), а в глубину. В-третьих, в модуль включен специальный пакет, позволяющий как раз оценить вклад отдельных фичей.

In [36]:
# Parameters for LGBoost
params = {'boost': 'gbdt',
          'feature_fraction': 0.05,
          'learning_rate': 0.01,
          'max_depth': -1,  
          'metric':'auc',
          'min_data_in_leaf': 50,
          'num_leaves': 32,
          'num_threads': -1,
          'verbosity': 1,
          'objective': 'binary'
         }

In [37]:
import time
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from catboost import CatBoostClassifier

In [38]:
def train_model(X, X_test, y, params, folds, model_type='lgb', plot_feature_importance=False, averaging='usual', model=None):
    oof = np.zeros(len(X))
    prediction = np.zeros(len(X_test))
    scores = []
    feature_importance = pd.DataFrame()
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y)):
        print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X.loc[train_index], X.loc[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]
        
        if model_type == 'lgb':
            train_data = lgb.Dataset(X_train, label=y_train)
            valid_data = lgb.Dataset(X_valid, label=y_valid)
            
            model = lgb.train(params,
                    train_data,
                    num_boost_round=20000,
                    valid_sets = [train_data, valid_data],
                    verbose_eval=1000,
                    early_stopping_rounds = 200)
            
            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test, num_iteration=model.best_iteration)
            
        if model_type == 'xgb':
            train_data = xgb.DMatrix(data=X_train, label=y_train, feature_names=X_train.columns) # для ускорения подсчета
            valid_data = xgb.DMatrix(data=X_valid, label=y_valid, feature_names=X_train.columns) # для ускорения подсчета

            watchlist = [(train_data, 'train'), (valid_data, 'valid_data')]
            model = xgb.train(dtrain=train_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200, verbose_eval=500, params=params)
            y_pred_valid = model.predict(xgb.DMatrix(X_valid, feature_names=X_train.columns), ntree_limit=model.best_ntree_limit)
            y_pred = model.predict(xgb.DMatrix(X_test, feature_names=X_train.columns), ntree_limit=model.best_ntree_limit)
        
        if model_type == 'sklearn':
            model = model
            model.fit(X_train, y_train)
            y_pred_valid = model.predict_proba(X_valid).reshape(-1,)
            score = roc_auc_score(y_valid, y_pred_valid)
            # print(f'Fold {fold_n}. AUC: {score:.4f}.')
            # print('')
            
            y_pred = model.predict_proba(X_test)[:, 1]
            
        if model_type == 'glm':
            model = sm.GLM(y_train, X_train, family=sm.families.Binomial())
            model_results = model.fit()
            model_results.predict(X_test)
            y_pred_valid = model_results.predict(X_valid).reshape(-1,)
            score = roc_auc_score(y_valid, y_pred_valid)
            
            y_pred = model_results.predict(X_test)
            
        if model_type == 'cat':
            model = CatBoostClassifier(iterations=20000, learning_rate=0.05, loss_function='Logloss',  eval_metric='AUC', **params)
            model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=[], use_best_model=True, verbose=False)

            y_pred_valid = model.predict_proba(X_valid)[:, 1]
            y_pred = model.predict_proba(X_test)[:, 1]
            
        oof[valid_index] = y_pred_valid.reshape(-1,)
        scores.append(roc_auc_score(y_valid, y_pred_valid))

        if averaging == 'usual':
            prediction += y_pred
        elif averaging == 'rank':
            prediction += pd.Series(y_pred).rank().values   #не дошли руки до ранговой классификации
        
        if model_type == 'lgb':
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = X.columns
            fold_importance["importance"] = model.feature_importance()
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    prediction /= n_fold
    
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    
    # Визуализируем данные по важности фичей
    if model_type == 'lgb':
        feature_importance["importance"] /= n_fold
        if plot_feature_importance:
            cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
                by="importance", ascending=False)[:50].index

            best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

            plt.figure(figsize=(16, 12));
            sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False));
            plt.title('LGB Features (avg over folds)');
        
            return oof, prediction, feature_importance
        return oof, prediction, scores
    
    else:
        return oof, prediction, scores

Подготовим данные, убрав признаки, которые точно не собираемся оценивать относительно их вклада в предсказание. Для этого напишем функции для оптимизации процесса дропа.

In [39]:
def excl_calc(data, team, feature):
    columns = [f"{team}{i}_{feature}" for i in range(1,6)]
    data.drop(columns, axis = 1, inplace = True)

In [40]:
new = data2.copy()
new_tst = data_tst2.copy()
for feat in player_feat:
    for t in ['r', 'd']:
        excl_calc(data = new, team = t, feature = feat)
        excl_calc(data = new_tst, team = t, feature = feat)

In [41]:
new.to_csv('df_train_filtered.csv', index = True)
new_tst.to_csv('df_test_filtered.csv', index = True)

In [ ]:
new = pd.read_csv("../input/df_train_filtered.csv", 
                                    index_col="match_id_hash")
new_tst = pd.read_csv("../input/df_test_filtered.csv", 
                                   index_col="match_id_hash")

In [42]:
x = new.reset_index(drop=True)
y = df_train_targets['radiant_win']
#x_tr, x_tst, y_tr, y_tst = train_test_split(x, y,test_size=0.3,random_state=SEED)
x_test = new_tst.copy().reset_index(drop=True)
n_fold = 5
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)

oof_lgb, prediction_lgb, scores = train_model(x, x_test, y, params=params, folds=folds, model_type='lgb', plot_feature_importance=True)

Мы видим, что отобранные нами фичи вносят значимый вклад в предсказание. Но наибольшее влияние оказывают разница средне заработанного золота между командами и id 5 игрока из команды radiant. И в целом, id игрока в команде оказывает сильное влияние на выигрыш. Потому следует провести лейблинг.

Если мы также посмотрим на распределение частот скоров предсказания исходов, то увидим, что с моделью LGBM и текущим сетом признаков мы даже достигали 1.0

In [71]:
plt.hist(prediction_lgb, bins=40);
plt.title('Distribution of predictions')
plt.xlabel('AUC score of model predictions')
plt.ylabel('Frequency of scores')

In [43]:
y_test_pred = prediction_lgb

df_submission = pd.DataFrame({"radiant_win_prob": y_test_pred}, 
                                 index=new_tst.index)

In [45]:
chosen_feat = list(scores[(scores.fold == 4) & (scores.importance >= 80.0)].feature)
x = new[chosen_feat]
chosen_test = new_tst[chosen_feat]
y = df_train_targets['radiant_win']
x_tr, x_tst, y_tr, y_tst = train_test_split(x, y,test_size=0.3,random_state=SEED)

In [48]:
chosen_test.to_csv('df_best_pred_test.csv', index = True)
x.to_csv('df_best_pred_train.csv', index = True)

In [47]:
x

In [90]:
rfc = RandomForestClassifier(n_estimators=300, max_depth=8, min_samples_leaf=50, min_samples_split=32, n_jobs=-1, random_state=SEED)
cv_scores_extended = cross_val_score(rfc, x, y, 
                                     cv=cv, scoring="roc_auc", n_jobs=-1)


In [91]:
cv_scores_extended.mean()

In [49]:
submission_filename = "submission_{}.csv".format(
    datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
df_submission.to_csv(submission_filename)
print("Файл посылки сохранен, как: {}".format(submission_filename))

Теперь подумаем, какую модель выбрать. У нас задача - бинарно классифицировать. Для бинарной классификации хорошо работает линейная регрессия, когда датасет представлен числовыми переменными. Хотя фнукция линейной регрессии разряженная и в случае неоднозначного отнесения к той/иной группе мы не будем четко видеть такие отклонения (они будут грубо оценены как та/иная категория).

Помимо линейной регрессии мы могли бы использовать KNNClassifier, который как раз позволил бы отлавливать, на сколько наблюдения похожи на тот/иной класс. Но в случае больших данных градиент не будет эффективен и у нас повысится вероятность попасть в локальный минимум и не выйти из него.



In [52]:
x

In [54]:
exclude = ['r1_hero_id', 'r2_hero_id', 'r5_hero_id', 'd1_hero_id', 'd3_hero_id', 'd4_hero_id']
x_tr2 = x_tr.drop(exclude, axis = 1)
param_grid_lr = {
    'max_iter': [20, 50, 100, 200, 500, 1000],                      
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],   
    'class_weight': ['balanced'],
    #'C' : 10.0 ** np.arange(-5, 5)
}
grid1 = GridSearchCV(LogisticRegression(penalty='l2'), param_grid = param_grid_lr, scoring='roc_auc')
grid1.fit(x_tr2, y_tr)

In [55]:
grid1.best_params_

In [56]:
lr_cl = LogisticRegression(class_weight='balanced', penalty='l2', max_iter=100, solver='sag')
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=SEED)
cv_scores_lr = cross_val_score(lr_cl, x.drop(['r1_hero_id', 'r2_hero_id', 'r5_hero_id', 'd1_hero_id', 'd3_hero_id', 'd4_hero_id'], axis = 1), y, 
                                     cv=cv, scoring="roc_auc", n_jobs=-1)
cv_scores_lr.mean()

Видим, что логистическая регрессия хоть и повысила с выбранными фичами скор предсказания до ~0.81, но с её помощью нельзя учесть категориальные переменные (которые оказывают сильный эффект на команду победителя). Из-за чего, возможно, наша модель не так эффективна.

Попробуем также другой алгоритм, позволяющий классифицировать смешанные типы данных. Для таких целей часто используют ансамбли деревьев, хотя с точки зрения ресурсоемкости они, пожалуй, одни из самых ресурсозатратных (особенно с плохо оптимизированными гиперпараметрами) и легче всего переобучаются.

In [57]:
n_estimators = [200, 300, 500, 1000] # number of trees in the random forest
max_features = ['auto', 'sqrt'] # number of features in consideration at every split
max_depth = [2, 3, 4, 6, 8] # maximum number of levels allowed in each decision tree
min_samples_split = [2, 6, 10] # minimum sample number to split a node
min_samples_leaf = [1, 3, 4] # minimum sample number that can be stored in a leaf node

criter = ['gini', 'entropy']



In [58]:
random_grid = {'n_estimators': n_estimators,

'max_features': max_features,

'max_depth': max_depth,

'min_samples_split': min_samples_split,

'min_samples_leaf': min_samples_leaf,

'criterion': criter}

In [70]:
rfc = RandomForestClassifier()
CV_rfc = GridSearchCV(estimator=rfc, param_grid=random_grid, cv= cv, n_jobs=-1, scoring='roc_auc')
CV_rfc.fit(x_tr2, y_tr)

In [ ]:
CV_rfc.best_params_

In [72]:
rf_model = RandomForestClassifier(n_estimators=300, max_depth=8, min_samples_leaf=3, min_samples_split=6, max_features='auto', criterion='entropy', n_jobs=-1, random_state=SEED)
cv_scores_rf = cross_val_score(rf_model, x.drop(exclude, axis = 1), y, 
                                     cv=cv, scoring="roc_auc", n_jobs=-1)
cv_scores_rf.mean()

Что же. До энкодинга наших id героев деревья немного лучше стали предсказывать при оптимизированных параметрах. Теперь введем лейблы нашим категориальным переменным.

In [73]:
from sklearn.preprocessing import OneHotEncoder

In [64]:
full_df = x.copy()

players_r = [f'r{i}' for i in [1, 2, 5]]
players_d = [f'd{i}' for i in [1, 3, 4]]
hero_r = [f'{player}_hero_id' for player in players_r]
hero_d = [f'{player}_hero_id' for player in players_d]
ohe = OneHotEncoder(sparse = False)
# radiant
dummy = ohe.fit_transform(full_df[hero_r[0]].values.reshape(-1, 1))
for c in hero_r[1:]:
    ohe = OneHotEncoder(sparse = False)
    dummy += ohe.fit_transform(full_df[c].values.reshape(-1, 1))
        
df = pd.DataFrame(
dummy, columns=ohe.categories_, index=full_df.index)
full_df = pd.concat([full_df, df.add_prefix('r_hero_')], axis=1)
full_df.drop(columns=hero_r, inplace=True)

# dire
ohe = OneHotEncoder(sparse = False)
dummy = ohe.fit_transform(full_df[hero_d[0]].values.reshape(-1, 1))
for c in hero_d[1:]:
    ohe = OneHotEncoder(sparse = False)
    dummy += ohe.fit_transform(full_df[c].values.reshape(-1, 1))
        
df = pd.DataFrame(
dummy, columns=ohe.categories_, index=full_df.index)
full_df = pd.concat([full_df, df.add_prefix('d_hero_')], axis=1)
full_df.drop(columns=hero_d, inplace=True)
    

Посмотрим, как теперь изменится наша модель.

In [75]:
x_tr, x_tst, y_tr, y_tst = train_test_split(full_df, y,test_size=0.3,random_state=SEED)

In [67]:
rfc = RandoRandomForestClassifierestClassifier()
CV_rfc = GridSearchCV(estimator=rfc, param_grid=random_grid, cv= cv, n_jobs=-1, scoring='roc_auc')
CV_rfc.fit(x_tr, y_tr)

In [76]:

cv_scores_encoded = cross_val_score(rf_model, full_df, y, 
                                     cv=cv, scoring="roc_auc", n_jobs=-1)

In [77]:
cv_scores_encoded.mean()

Сохраним датасет, чтобы не получать все заново.

In [80]:
full_df.to_csv('df_train_encoded.csv', index = True)

Теперь начнем подбирать гиперпараметры и варьировать модель! Ух!

In [19]:
try:
    dataset = pd.read_csv('../input/dota2-competition-bi2022/df_train_encoded.csv', index_col = 0)
    x = dataset.values
    y = df_train_targets["radiant_win"].values.astype("int8")
    x_tr, x_tst, y_tr, y_tst = train_test_split(x, y,test_size=0.3,random_state=SEED)
except:
    x = full_df.values
    y = df_train_targets["radiant_win"].values.astype("int8")
    x_tr, x_tst, y_tr, y_tst = train_test_split(x, y,test_size=0.3,random_state=SEED)
    

In [24]:
n_estimators = [i for i in range(200, 1000, 100)] # number of trees in the random forest
max_features = ['auto', 'sqrt'] # number of features in consideration at every split
max_depth = [i for i in range(1, 11, 2)] # maximum number of levels allowed in each decision tree
min_samples_split = [2, 6, 10] # minimum sample number to split a node
min_samples_leaf = [1, 3, 4] # minimum sample number that can be stored in a leaf node

criter = ['gini', 'entropy']

In [25]:
random_grid = {'n_estimators': n_estimators,

'max_features': max_features,

'max_depth': max_depth,

'min_samples_split': min_samples_split,

'min_samples_leaf': min_samples_leaf,

'criterion': criter}

Так как еще при подборе фичей более сложные ансамбли, как LGBM, XGB, которые имеют ускоренные функции спуска и меньшую вероятность попадания в локальный минимум, показали себя лучше, то попробуем примениь XGB как на данных до энкодинга, так и с энкодингом.

In [81]:
from xgboost import XGBClassifier

xgb_cl = XGBClassifier(objective ='binary:logistic', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 2, alpha = 10, n_estimators = 100, n_jobs=8)


In [82]:
cv_scores_xgb_wo_enc = cross_val_score(xgb_cl, x.drop(exclude, axis = 1), y, 
                                     cv=cv, scoring="roc_auc", n_jobs=-1)
cv_scores_xgb_enc = cross_val_score(xgb_cl, full_df, y, 
                                     cv=cv, scoring="roc_auc", n_jobs=-1)

In [83]:
cv_scores_xgb_wo_enc.mean()

In [84]:
cv_scores_xgb_enc.mean()

К сожалению, не сильно изменило картины. Но в целом, проведенный инжениринг не только снизил размерность данных (что в целом увеличивает подъемность анализа), но также позволил более чувствительнее определять вероятность выигрыша при учете командных метрик. Мы также увидили, как разные алгоритмы ведут себя в зависимости от данных. И что та же логистическая регрессия может быть эффективна для такой задачи, но при определенном наборе признаков.

In [35]:
df_test_features